In my masters thesis I explored the use of tolerance limits to achieve appropriate coverage probabilities in the context of the A-basis and the B-basis of composite materials. I also demonstrated that samples are not required when calculating the coverage probabilities in a one-way ANOVA model. To provide numerical examples I created a script in R that calculates the coverage probabilities of the A-basis and B-basis under the assumption of a normal distripution.  I won't go into the necessity for using tolernace limits here, but (CITE HERE) are good reading if interested.

Let the breaking strength of a composite material be $X\sim(\mu,\sigma^2)$.  If we use a traditional represenative of the random variable such as the mean it isn't going to be very helpful.  Half the material would have a breaking strength less than the mean. This isn't very helpful to engineers trying to design aircraft parts so instead A-basis and B-basis are used.  \n",
    "If we assume a normal distribution for our random variable, which is often the case, then by Military Handbook 17F (2002) the lower limits of one-sided conﬁdence intervals for the 1st and the 10th percentiles of $X$ with conﬁdence coeﬃcient $95\%$ are referred to as A-basis and B-basis. The lower limits of a confidence interval are used  since the percentiles themselves fail to provide  the desired coverage probibities of $99\%$ and $90\%$.

Assuming $X\sim N(\mu,\sigma^2)$ and letting $d = n$ (n=sample size for each factor) and $f = N-k$ (N=total sample and k=number of factors in the ANOVA model), then the A-basis and B-basis coverage probabilities are given by,
$$\begin{equation}
        P(X\geq A)= P\left(t(f)\geq\frac{-K_A(d,f)}{\sqrt{1+\frac{1}{d}}}\right)=P\left(t(f)\geq\frac{-t_{0.95}(2.3263\sqrt{d},f)}{\sqrt{1+d}}\right)\\
        P(X\geq B)= P\left(t(f)\geq\frac{-K_B(d,f)}{\sqrt{1+\frac{1}{d}}}\right)=P\left(t(f)\geq\frac{-t_{0.90}(1.2816\sqrt{d},f)}{\sqrt{1+d}}\right)
\end{equation}
$$


The calculation of the coverage probabilities are pretty straight forward in R since The Military Handbook 17F provides us with an industry accepted estimation of $K_A$ and $K_B$.

In [18]:
#give values for d and f
#these are just examples, user would put in, play with if you like.

d=30
f=87
    
#using estimation from the military handbook 17f for K values
    
x = 1/sqrt(f)
q = 1 - 2.327*x + 1.138*x^2 + 0.6057*x^3 - 0.3287*x^4
b_A = 2.0643*x - 0.95145*x^2 + 0.51251*x^3
c_A = 0.36961 + 0.0026958*x - 0.65201*x^2 + 0.011320*x^3
b_B = 1.1372*x - 0.49162*x^2 + 0.18612*x^3
c_B = 0.36961 + 0.0040342*x - 0.71750*x^2 + 0.16963*x^3
    
K_A = 2.3263/sqrt(q)+sqrt((1/(d*c_A))+(b_A/(2*c_A))^2)-(b_A/(2*c_A))
K_B = 1.2816/sqrt(q)+sqrt((1/(d*c_B))+(b_B/(2*c_B))^2)-(b_B/(2*c_B))

K_A
K_B

[1] 2.790196

[1] 1.648513

Now that we have values for K_A and K_B we can easily get the coverage probability using the pt fuction built into R.

In [19]:
#using the K_A and K_B values to calculate the coverage probability using the pt function\n",

cov_A = pt(-K_A/(sqrt(1+(1/d))),f,lower.tail=FALSE)
cov_B = pt(-K_B/(sqrt(1+(1/d))),f,lower.tail=FALSE)
    
cov_A
cov_B

[1] 0.996323

[1] 0.9457575

Now we might be wondering if using the estimate for K and the built in pt function provides an accurate enough value for the coverage probility.  This can check this by using Simpson's rule to make the same calculation.  

In [20]:


#first we need to create a function for the pdf of the t-distribution
#we a going to use the built in gamma function to do this
    
        
pdf_T=function(f,x){
    r1=gamma((f+1)/2)         #seperating the function for clarity
    r2=(sqrt(f*pi))*gamma(f/2)
    r3=(1+(x^2/f))^-((f+1)/2)
        
    (r1/r2)*r3
    }
    
    #now we can use simpson's rule to calculate P(0=<t(f)<=x)
    
    T_ox=function(f,x){
        
        n=10000
        h=x/(2*n)
        
        sum0=pdf_T(f,0)
        
        i = seq(1,2*n-1,2)        #R's ease with vectors is great, no loops
        sum1=sum(4*pdf_T(f,h*i))  #this would be slow otherwise    
        
        j = seq(2,2*n-2,2)
        sum2=sum(2*pdf_T(f,h*j))
        
        sum3=pdf_T(f,x)
        
        sumtotal=(h/3)*(sum0+sum1+sum2+sum3)
        
        return(sumtotal)
    
    }
simp_cov_A=.5+T_ox(f,K_A/(sqrt(1+(1/d))))   #adding the .5 to cover the negative part.
simp_cov_B=.5+T_ox(f,K_B/(sqrt(1+(1/d))))

simp_cov_A
simp_cov_B

[1] 0.996323

[1] 0.9457575

In [22]:
#Now lets look at the percent difference

diff_A=cov_A-simp_cov_A

ave_A=(cov_A+simp_cov_A)/2

percent_diff_A=(diff_A/ave_A)*100

diff_B=cov_B-simp_cov_B

ave_B=(cov_B+simp_cov_B)/2

percent_diff_B=(diff_B/ave_B)*100

percent_diff_A
percent_diff_A

[1] -3.565825e-13

[1] -3.565825e-13

So thats really, really close. Of course we already new this. R function are very reliable in there accuracy.  Except when they are not.  You may have noticed that K_A and K_B can be calculated using the non-centralized version of pt.